In [6]:
import numpy as np #importa biblioteca Numpy
import pickle #importa a biblioteca pickle, usada pra salvar arquivos
file_path = "tabela.pickle" #nome do arquivo que ficará salvado

In [7]:
class probabilidade_dados: # Cria a classe
  """

  Attributes:
    numero_dados:
    tamanho_dados:
    tamanho_min:
    tamanho_max:
    porcentagem:
    valores_possiveis:
    soma_porcentagem:
    treinamento:
  """
  def __init__(self, numero_dados= 1, tamanho_dados= 1, tamanho_min= 0, tamanho_max = (2**64), porcentagem = False, soma_porcentagem= False, treinamento = False):
  # Inicia a classe com suas variaveis


    self.numero_dados_max = int(10) # variavel de mudança pesada: Numero de todos os dados que serão feitos nas tabelas
    self.tamanho_dados_max = int(10) # variavel de mudança pesada: Tamanho de todos os dados que serão feitos nas tabelas
    #Durante o treinamento todos esses valores serão utilizados calculados e postos na tabela

    self.treinamento = treinamento # Flag pra saber se queremos criar a variavel pela primeira (True) vez ou conferir os valores (False)

    #Define a numero que sera visto na colune representante do numero de dados
    #Esse numero não pode ser maior do que o numero maximo de dados
    #Caso seja maior utilizaremos o valor maximo
    if numero_dados > self.numero_dados_max:
      print("O numero maximo de dados é %s, logo utilizaremos esse numero nos calculos" % self.numero_dados_max)
      self.numero_dados = self.numero_dados_max
    else:
      self.numero_dados = int(numero_dados)

    #Define a numero que sera visto na colune representante do tamanho de dados
    #Esse numero não pode ser maior do que o tamanho maximo de dados
    #Caso seja maior utilizaremos o valor maximo
    if tamanho_dados > self.tamanho_dados_max:
      print("O tamanho maximo dos dados é %s, logo utilizaremos esse numero nos calculos" % self.tamanho_dados_max)
      self.tamanho_dados = self.tamanho_dados_max
    else:
      self.tamanho_dados = int(tamanho_dados)

    self.tamanho_min = int(tamanho_min) # Tamanho minimo cuja propabilidade queremos avaliar. Se não dito seu valor será 0

    #Verifica se a variavel Tamanho Maximo é maior do que o numero maximo alcançavel pelo dado
    #se não for, o Tamanho Maximo se torna o maior numero possivel
    #Tamanho maximo cuja probabilidade queremos avaliar
    #Caso o usuario não passe um parametro esse valor está em um numero absurdo
    #para que entre no if e se torne igual ao maior numero alcançavel
    if tamanho_max >= self.numero_dados*self.tamanho_dados:
      if not(self.treinamento):
        print("O tamanho maximo que podemos avaliar com essa composição de %s dados de tamano %s é %s" % (self.numero_dados, self.tamanho_dados, self.numero_dados*tamanho_dados))
      self.tamanho_max = self.numero_dados*self.tamanho_dados #maior numero alcançavel é igual a quantidade de lados de cada dado vezes a quantidade de dados rolados
    else:
      self.tamanho_max = int(tamanho_max)

    self.porcentagem=porcentagem #verifica se queremos o valor rm forma de porcentagem
    self.soma_porcentagem= soma_porcentagem #verifica se queremos a soma da probabilidade de diversos valores

    #Devido ao numero de contas utilizadas o codigo pode se tornar lento com o aumento do numero e tamanhos de dados usados
    #Logo treinamos o codigo e salvamos a tabela que armazena as contas pra cada combinação de numero de dados e tamanho de dados
    #Caso queiramos apenas utiliza-la, lemos os dois primeiros indices dessa taela, para identificar o ponto desejado e avaliamos este vetor
    if treinamento: #Caso queiramos treinar
      tabela_valores= self.criar_tabela_valores() # Calculamos toda a tabela. Operação lenta

      # Salvamos a tabela em tabela.pickle
      with open(file_path, 'wb') as file:
        pickle.dump(tabela_valores, file)
        print("Tabela de Valores das somas de até %s dados de até %s tamanho, salvas em %s"%(self.numero_dados_max,self.tamanho_dados_max,file_path))
    else: # Caso queiramos ler a tabela

      # Carregamos a tabela do ponto salvo
      loaded_tabela = None
      with open(file_path, 'rb') as file:
        loaded_tabela = pickle.load(file)

        #Calculamos o valor de acordo com o ponto pedido
        self.calcular_soma(tabela_valores= loaded_tabela ,numero_dados= self.numero_dados, tamanho_dados= self.tamanho_dados,tamanho_min= self.tamanho_min,tamanho_max= self.tamanho_max, porcentagem= self.porcentagem, soma_porcentagem = self.soma_porcentagem)

  def calcular_numero_somas(self, numero_dados, tamanho_dados, n_min, n_max): #Função para calcular as possiveis somas daquela composição de dados

    #vetor que armazena todos os valores possiveis
    #o maior valor possivel é caso a rolagem gere todos os maiores valores em todos os lados
    #o valor então é somado
    #por exemplo, 24 em 3 dados de 8 lados
    #Adicionamos +1 pois levamos em consideração a possibilidade do 0
    valores_possiveis = np.zeros((numero_dados*tamanho_dados)+1)

    #matriz de colunas quase iguais, caa coluna passa de 1 até o valor maximo do dado
    #e tem linhas iguais a quantidade de dados rolados
    #a ultima linha da matriz, inicializa-se sempre como 0
    #ela existe para identificar quais valores serão utilizados em uma soma
    #por exemplo se a ultima linha for [-1, 0, -3]
    #isso significa que o programa somara os itens [0, 1] + [1, 0] + [2, 3]
    matriz_dados = np.append(np.array(range(1, tamanho_dados+1)), 0)
    if (numero_dados-1) == 0:
      matriz_dados = np.matrix(matriz_dados).T #Alteração necessaria para o caso inicial onde o tamanho seria pequeno demais
    else:
      for i in range(numero_dados-1):
        matriz_dados = np.column_stack((matriz_dados, np.append(np.array(range(1, tamanho_dados+1)), 0)))

    aux = tamanho_dados #auxiliar para facilitar os calculos e a apresentação do programa

    #a primeira soma guiada pela Matriz Dados possiveis é feita antes de entrarmos nos laços de repetição
    valores_possiveis[np.sum(matriz_dados[0,:])] += 1

    continua = True # flag para continuarmos o laço de repetição inicial
    while(continua):
      soma = 0 # variavel soma é zerada
      k = numero_dados-1 # variavel k inicia-se com o numero da ultima coluna de Matriz Dados

      while(k>=0):
        #verificamos se o ultimo elemento da coluna avaliada representa o maior valor tiravel pelo dado
        #e assim da ultima linha da parte que desejamos avaliar da Matriz dado
        #para evitar problemas matematicos todos os valores da ultima linha são <=0
        #uma vez que um dado usual só tem lados positivos
        #logo todos os valores da ultima linha, e suas consequentes analises são <=0

        if matriz_dados[aux, k] == (tamanho_dados)*(-1):
          #verificamos se TODOS os valores da ultima linha indicam o fim daquela coluna
          #dessa forma teremos terminado a matriz e deveremos parar o laço

          if (matriz_dados[aux, :] == (tamanho_dados)*(-1)).all():

            continua = False
            k = -1
          #Caso continuemos os laçoes de repetição e essa coluna tenha chegado ao seu fim
          #Re-inicializamos o ultimo valor da coluna analisada
          #e fazemos a mesma analise na coluna anterior
          else:

            matriz_dados[aux, k] = 0
            k -= 1
        else:
          #Caso aquela coluna não tenha chegado ao seu fim, diminuimos o valor
          #de sua ultima linha em 1, para testes futuros

          matriz_dados[aux, k] -= 1
          k = -5 # k é mudado para um valor < 0, de forma a sairmos do laço de repetição

          #If utilizado para que a existência da ultima linha seja ignorada durante a
          #quantidade de somas
          #So entramos no If se NÃO HOUVER NENHUM valor da igual ao valor maximo
          #negativo alcançavel por ela
          #dessa forma o vator da matriz, que existe para apontar para quais valores
          #do resto da matriz deva ser utilizado, não possa apontar para um valor
          #dentro da linha de controle
          if not((matriz_dados[-1,:] == (tamanho_dados)*(-1)).any()):

            #todos os valores apontados pela linha de controle são somados
            for i in range(numero_dados):
              soma += matriz_dados[(-1*matriz_dados[aux, i]), i]

            #O vetor valores_possiveis e atualizado para representar que existe mais uma
            #soma que resulte no valor daquele index
            valores_possiveis[soma] += 1

    return valores_possiveis #Retorna o vetor valores possiveis

  def criar_tabela_valores(self): # Função para calcular toda a tabela de valores possiveis

    #Vetor inicial da tabela valores
    #Iguala o tamanho da tabela de valores mais os dois numeros inicias que tageiem aquele vetor
    #logo é igual a ((numero dados * tamanho dados) +1) +2
    tabela_valores = np.zeros((self.numero_dados_max*self.tamanho_dados_max)+3)

    # Variavel auxiliar para construção da tabela
    aux_tabela_valores = tabela_valores

    #Laços de repetição para criação da tabela. Nessa composição passamos inicialmente o tamanho dos dados e então o numero de dados
    #Isso se da porque o aumento no numero de dados acarreta no maior aumento do tamanho do tempo de programa
    for i in range(1, self.numero_dados_max+1):
      #tabela valores inicialmente é um vetor
      tabela_valores[0] = i # o primeiro valor do vetor é tageado de acordo com o numero de dados utilizado

      for j in range(1, self.tamanho_dados_max+1):
        tabela_valores[1] = j # o segundo valor do vetor é tageado de acordo com o tamanho dos dados utilizados

        #calculamos o numero de somas possiveis
        numero_somas = self.calcular_numero_somas(numero_dados= i, tamanho_dados= j, n_min= 0, n_max= self.tamanho_dados_max*self.tamanho_dados_max)
        #garantimos que os vetores menores serão transformados ou "padded" para serem do mesmo tamanho dos maiores
        #os valores que são padded são sempre 0
        #os dois primeiros valores, que representam as taggs também são padded com o valor 0
        numero_somas = np.pad(numero_somas, [2, ((self.numero_dados_max*self.tamanho_dados_max)+1)-numero_somas.size])

        #o vetor tabela valores tem o vetor numero_somas adicionado a ele
        tabela_valores +=  numero_somas
        # A variavel aux_tabela_valores se torna uma matriz
        #a cada interação o vetor tabela_valores calculados é adicionado como a ultima linha da matriz
        aux_tabela_valores = np.vstack([aux_tabela_valores, tabela_valores])
        tabela_valores[1:] *= 0 #os valores de tabela valores, exceto suas tags são zerados

    tabela_valores = aux_tabela_valores # o valores de aux são passados para tabela valores
    del aux_tabela_valores # a variavel aux é destruida pra gerar mais espaço de programa
    #print(tabela_valores) # possibilidade de imprimir tabela valores caso queira ve-la
    return tabela_valores # devolvemos a tabela de valores


  def calcular_soma(self, tabela_valores, numero_dados, tamanho_dados, tamanho_min, tamanho_max, soma_porcentagem, porcentagem):
  # função para calcular a soma.
  # de acordo com o tamanho dos dados pedidos, o numero de dados pedidos e a faixa de valores que desejamos ver
  # o resultado pode ser a soma da faixa de valores ou a apresentação de cada
  # o resultado pode ser em porcentagem ou valores aritmeticos

    for i in tabela_valores: # pegamos cada vetor da tabela
      if i[0] == numero_dados and i[1] == tamanho_dados: # avaliamos as tags inicias do vetor [numero_dados, tamanho_dados]
        count = tamanho_min # variavel utilizada para apresentação apropriada, inicia no valro minimo é é acrescida quando necessario
        if soma_porcentagem: # Se o resultado é a soma
          if porcentagem: # Se o resultado é em porcentagem
            # Calculamos todas as somas dos valores do vetor i. Ignorando dos 2 primeiros itens, que são tags
            # Multiplicamos a soma por 100 para representar porcentagem
            # Dividimos pelo numero total de contas utilizado, que é tamanho_dados**numero_dados
            print("Chance do resultado ser entre %s e %s é: %s%%"%(tamanho_min, tamanho_max, np.sum(i[2+tamanho_min:2+(tamanho_max+1)])*100/(tamanho_dados**numero_dados)))
          else: # se o resultado não é porcentagem
            # Calculamos todas as somas dos valores do vetor i. Ignorando dos 2 primeiros itens, que são tags
            # Dividimos pelo numero total de contas utilizado, que é tamanho_dados**numero_dados
            print("Chance do resultado ser entre %s e %s é: %s"%(tamanho_min, tamanho_max, np.sum(i[2+tamanho_min:2+(tamanho_max+1)])/(tamanho_dados**numero_dados)))
        else: # se queremos o valor de cara possibilidade dentro da faixa
          for j in i[2+tamanho_min:2+(tamanho_max+1)]: # corremos o vetor avaliado, ignorando os seus dois primeiros itens, pois são tags
            if porcentagem: # se o resultado é em porcentagem
              # Multiplicamos a soma por 100 para representar porcentagem
              # Dividimos pelo numero total de contas utilizado, que é tamanho_dados**numero_dados
              #Apresentamos 1 valor por vez
              print("%s: %s%%" % (count, j*100/(tamanho_dados**numero_dados)))
            else: # se o resultado não é em porcentagem
              # Dividimos pelo numero total de contas utilizado, que é tamanho_dados**numero_dados
              #Apresentamos 1 valor por vez
              print("%s: %s" % (count, j/(tamanho_dados**numero_dados)))

            count += 1 # cunt é acrescido




In [8]:
probabilidade_dados(treinamento= True)

Tabela de Valores das somas de até 5 dados de até 5 tamanho, salvas em tabela.pickle


In [9]:
probabilidade_dados(numero_dados= 3, tamanho_dados=4, tamanho_min= 8, tamanho_max= 10, porcentagem=True, soma_porcentagem= True, treinamento= False)

Chance do resultado ser entre 8 e 10 é: 43.75%
